In [2]:
# まずはrandom seedを固定する（torch.backends...の方は，ネットワークやロス計算，勾配更新の際の確率的な挙動を，決定論的に行うためのoption?）
import torch
from torchtext import data
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)

In [3]:
# torchtextではNLPでよく使われるデータセットは容易に取得可能になっている
from torchtext import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [4]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of training examples: {len(test_data)}')

Number of training examples: 25000
Number of training examples: 25000


In [5]:
print(vars(train_data.examples[0]))

{'text': ['For', 'a', 'movie', 'that', 'gets', 'no', 'respect', 'there', 'sure', 'are', 'a', 'lot', 'of', 'memorable', 'quotes', 'listed', 'for', 'this', 'gem', '.', 'Imagine', 'a', 'movie', 'where', 'Joe', 'Piscopo', 'is', 'actually', 'funny', '!', 'Maureen', 'Stapleton', 'is', 'a', 'scene', 'stealer', '.', 'The', 'Moroni', 'character', 'is', 'an', 'absolute', 'scream', '.', 'Watch', 'for', 'Alan', '"', 'The', 'Skipper', '"', 'Hale', 'jr', '.', 'as', 'a', 'police', 'Sgt', '.'], 'label': 'pos'}


In [6]:
import random
train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [7]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of test examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of test examples: 25000


- dataは，単語のone-hot-vectorとして扱う．
- そのため，このベクトルのdimentionは単語の種類数になる．このデータセットの単語の種類数は，100,000．
- 多すぎて，GPUに乗せることが困難->そこで，このベクトルの次元を圧縮する方法が2つある
- 一つは，出現回数を上位何件で区切る．もう一つは，出現回数が一定数以上のものを使用する．（同じこと言ってね？？？）
- ここでは，前者の方法を使う．
- また，ここでは，意味わからない単語（辞書に載ってない単語）は，"unk"に変換する

In [8]:
MAX_VOCAB_SIZE = 25_000
TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [9]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


- この2は何かと言うと，先ほど定義した'unk'と'pad'
- padは，文章が短い時があるので，その時固定長のベクトルにするために，埋めるための値

In [10]:
# よく使われている単語top 20
print(TEXT.vocab.freqs.most_common(20))

[('the', 202475), (',', 192116), ('.', 165085), ('a', 109230), ('and', 109174), ('of', 101087), ('to', 93504), ('is', 76396), ('in', 61292), ('I', 54006), ('it', 53321), ('that', 48902), ('"', 44039), ("'s", 43236), ('this', 42363), ('-', 37002), ('/><br', 35684), ('was', 34977), ('as', 30125), ('with', 29740)]


In [11]:
# itos it string to int
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']


In [12]:
# stoi is int to string
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x118014e18>, {'neg': 0, 'pos': 1})


BucketIteratorは，自動で最小数のpadでpaddingしてくれた文章を流してくれる特殊なdataloader

In [13]:
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device
    )

In [14]:
import torch.nn as nn
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        # text = [sent len, batch size]
        
        # embedding layerでは，textのままだと次元数が多すぎるので，それを削減するために，
        embedded = self.embedding(text)
        # embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        # output = [sent len, batch size, hid dim]
        # hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1, :, :], hidden.squeeze(0))
        return self.fc(hidden.squeeze(0))
        

- input dimentionは単語数
- embedding dimensionはone-hot-vectorというsparseなデータを圧縮したもので，単語数に依存するが，だいたい50-25-が通常
- output dimensionは分類のクラス数だが，今回は2なので，scalar

In [15]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

Count the number of trainable parametes

In [16]:
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_params(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


## Train the Model

Define the optimizer.
This case use "SGD"

In [17]:
import torch.optim as optim
optimizer = optim.SGD(model.parameters(), lr=1e-3)

Define the loss function which is called "criterion" in PyTorch.  
This case use "binary cross entropy with logits"

In [18]:
criterion = nn.BCEWithLogitsLoss()

place the model and the criterion to the GPU

In [19]:
model = model.to(device)
criterion = criterion.to(device)

calcurate the accuray of the model's precictions

In [20]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right,
    this returns 0.8, NOT 8
    """
    
    # round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    
    return acc

- model.train()はモデルをトレーニングモードにするためのもので，トレーニングモードにすると，DropoutやNormalization等が有効になる．
- gradientは毎iterationでzeroにする．（前回の更新時の値が入ってしまっているため
- modelのforward computationはmodel(input)だけで大丈夫．
- また，出力は\[batch_size\]の形になるように整形する
- loss.backward()でgradientを計算して，optimizer.step()で，modelおparametersをupdateする
- loss.item()はget only the valueの時に使われる．
- batch.labelはlossを計算するために，何処かのタイミングでfloatにする必要がある

In [21]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss /len(iterator), epoch_acc / len(iterator)

- model.eval()はtraining modeとは逆に，モデルを評価モードにするためのmethod
- with no_grad()の中では，計算グラフが作られなくなる．

In [22]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Calcurate the training times

In [23]:
import time

def epoch_time(st, et):
    elapsed = et - st
    elapsed_mins = int(elapsed / 60)
    elapsed_secs = int(elapsed - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [25]:
N_EPOCHS = 5
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    st = time.time()
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    et = time.time()
    epoch_mins, epoch_secs = epoch_time(st, et)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), './models/tut1-model.pth')
        
    print(f'Epoch: {epoch + 1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
    print(f'\tVal. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc * 100:.2f}%')

Epoch: 01 | Epoch Time: 10m 4s
	Train Loss: 0.693 | Train Acc: 49.81%
	Val. Loss: 0.697 | Val. Acc: 49.75%
Epoch: 02 | Epoch Time: 9m 27s
	Train Loss: 0.693 | Train Acc: 49.90%
	Val. Loss: 0.697 | Val. Acc: 49.75%
Epoch: 03 | Epoch Time: 10m 7s
	Train Loss: 0.693 | Train Acc: 49.95%
	Val. Loss: 0.697 | Val. Acc: 49.17%
Epoch: 04 | Epoch Time: 9m 36s
	Train Loss: 0.693 | Train Acc: 50.20%
	Val. Loss: 0.698 | Val. Acc: 51.04%
Epoch: 05 | Epoch Time: 9m 43s
	Train Loss: 0.693 | Train Acc: 49.91%
	Val. Loss: 0.697 | Val. Acc: 49.15%


- lossもそんなに低くなっていないし，accuracyもそこまで高くなってない
- これは，ここで使用しているモデルや最適化手法等が簡単のために性能の良いものが選択されていないことによります．そのため，これからmodelをimproveしていく

In [27]:
model.load_state_dict(torch.load('./models/tut1-model.pth'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.710 | Test Acc: 47.13%


## Next Steps
- packed padded sequences
- pre-trained word embeddings
- different RNN architecture
- bidirectional RNN
- multi-layer RNN
- regularization
- a different optimizer